## 2.2. <a id='toc2_2_'></a>[Environment Setup](#toc0_)

The environment set for the thymio includes the goal, static obstacles, background and thymio covering. The choices we made for the setup are highly related to the vision part as we have to be able to process the image captured. We decided the static obstacles would be in 2D to avoid shadows, that way they are easier to detect with the camera. The obstacles are black on a lighter background (ideally white but it would work with any light color). Our obstacles have to be shapes with sharp lines and edges, they cannot be curvilinear. We are using a red square goal and two green and blue points on the thymio. Those are the features of our environment. The setup has no boundary points and extend to the full viewing angle of the camera, so that the full picture detected can be exploited. 

A further explanation on how those obstacles and features are detected is given in the computer vision part. 

(Picture of an example of an environmemt setup) 


## 3.1. <a id='toc3_4_'></a>[Computer Vision](#toc0_)

<u>**Idea**</u>

The **Vision** module is mostly based on the OpenCV documemtation. Other sources used are mentionned throughout the explanation. 
The purpose of the computer vision part, which is autonomous from the other parts but is essential to compute the reference points of the setup, is to capture and use the picture from the camera to extract the main features of the environment, analyze them and give the needed infos to the global navigation and odometry modules (i.e. actual robot position and angle, goal position and obstacles positions). The particularity of the Computer Vision module is that it does not commumicate with the robot. 

The selected approach is based on color, shape and corners recognition. We began with "perfect" pictures of setup created virtually and had to rescale quite a bit the values once we switched to the "noisy" pictures of the camera.

As for the code implememted, we created a vision class containing the basic arguments needed and computed by vision and the functions that just have to be called in the main yhen needed. Some vision infos need to be recomputed each time, such as the robot position and angle, and some of them need only be computed at first step, like the obstacles position and corners that yill not move during an execution. 



**1. Image capturing and filtering**

At first, a frame is extracted from the camera. Note that we capture two frames because we noticed sometimes, at first connection of the camera to the computer, the first frame captured was "yellowish" and too bright to be used. We then filter the image several times to avoid noise and have a "smooth" picture to perform the other steps on. 

For the filtering, we tested several combinations and the one in the code beloy seemed to be the one that worked best. We therefore performs 2D linear filtering, then we blur the frame and then apply an additional median blur. Our filtered frame is registered in the vision Class to be used by all other vision functions. 

In [ ]:
def capture_image(self,cap): 
        #capture a frame out of the video that will be used through CV part 
        ret, self.frame = cap.read()
        ret, self.frame = cap.read()
        kernel = np.ones((5,5),np.float32)/25
        img = cv2.filter2D(self.frame,-1,kernel)
        img = cv2.blur(img,(5,5))
        self.img = cv2.medianBlur(img,5)


**2. Goal position, thymio position and angle tracking**

To find the coordinates of the goal and start position, we use color filtering. We apply a color mask of the target color (red for the goal, blue and green for the robot) on the picture and then use maxLoc function or contour finding to extract the coordinates. The values of the color mask have been found using hsv color space graph and forums, they are empiric values adapted to our setup, lightning and the colors of the papers we are using. Indeed, one of the main "flaws" of the vision part, thaht will be discussed more in detail in the weakness part, is that it is quite sensitive to light and color changes and uniformity.

The function find_goal_pos is applying a red mask to the filtered frame, then we apply a contour detection on the thresholded image and find the moment, i.e. center of the shape. This value corresponds to the goal position. 

The angle of the thymio is computed using the coordinates of the green and blue point on top of the thymio. We obtain a positive angle going from zero to two py starting from the x axis and turning counterclockwise. 

At the end of the functions computing the positions and angle, we store those values in the shared Thymio Class to be used by everybody.

The *find_angle* function pasted below computes the coordinates of the blue point on the thymio at first in the same way the coordinates of the green (=start point) are computed. Then, using both green (back) and blue (front) coordinates, we compute the angle. 

Below is the thresholded image of the frame given as example with red mask applied. 


In [1]:
def find_angle(self,robot):
        hsv = cv2.cvtColor(self.img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, self.LOW_BLUE, self.HIGH_BLUE)
        blue = cv2.bitwise_and(self.frame,self.frame, mask = mask)
        gray = cv2.cvtColor(blue, cv2.COLOR_RGB2GRAY) 
        _, self.thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY) 
        (minVal, maxVal, minLoc, maxLoc) = cv2.minMaxLoc(self.thresh1)
        [x_front,y_front] = maxLoc
        if (x_front != 0) or (y_front != 0):
            [self.x_front,self.y_front] = [x_front,y_front]
            if self.y_front < self.y_back : 
                if self.x_front > self.x_back : 
                    self.teta = np.arccos((self.x_front-self.x_back)/(np.sqrt(np.power((self.x_front-self.x_back),2)+np.power((self.y_front-self.y_back),2))))
                if self.x_front <= self.x_back : 
                    self.teta = np.pi - np.arccos((self.x_back-self.x_front)/(np.sqrt(np.power((self.x_front-self.x_back),2)+np.power((self.y_front-self.y_back),2))))
            if self.y_front >= self.y_back : 
                if self.x_front > self.x_back : 
                    self.teta = 2*np.pi - np.arccos((self.x_front-self.x_back)/(np.sqrt(np.power((self.x_front-self.x_back),2)+np.power((self.y_front-self.y_back),2))))
                if self.x_front <= self.x_back : 
                    self.teta = np.pi + np.arccos((self.x_back-self.x_front)/(np.sqrt(np.power((self.x_front-self.x_back),2)+np.power((self.y_front-self.y_back),2))))
            robot.setPositions(self.x_back,self.y_back,self.x_goal,self.y_goal,self.teta)
            robot.setVisionDone(True)
        #Hidden thymio, do not update the vision variables 
        else:
            robot.setVisionDone(False)


**3. Obstacles**

For the obstacles identification part, it depends on the type of global navigation used. In the case of visibility graph, the process used is the following: 
At first, we apply a big thresholding on the gray image to obtain a frame containing only the obstacles. The corners and moments of the obstacles are obtained using openCV corners and moments finding functions. We implememted a small part of code placing the corners further away from the shape to avoid the thymio running into the obstacles when following the side of a shape. The value of the distance of the corners to their shape is a variable that can be changed in the code. We also take out the corners too close to the borders as we do not want the thymio to go outside the setup boundaries. Those methods are implememted in the function *find_corners*. 

Then we have to provide the global navigation part a "visibility graph" with the nodes and weights of each node to all others. Obviously the nodes considered are the corners plus start and goal position. We therefore compute a matrix containing the distance of each point to all others, implememted in the *compute_dist_mx* function. But we cannot just calculate the distance between points because if there is an obstacle in the way between two points, it must not be taken into account. We therefore decided to use threshold on the line to avoid this. Here is a quick explanation of the approach: on the thresholded image of the obstacles, if we extract the pixel values of the line between two points, if it contains black, then there is an obstacle in the way and we do not compute the distance. 

Before calling compute_dist_mx function, we therefore call the function *trace_contours* that traces the lines beteween the corners to avoid computing path in the "forbidden area" between obstacles and corners. 

Below is a picture of the frame used for this setup to compute the distance matrix. 

In [ ]:
def find_corners(self):
        gray = cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY) 
        ret, thresh2 = cv2.threshold(gray, self.BLACK_THRESHOLD, 255, cv2.THRESH_BINARY) 
        contours, _ = cv2.findContours( 
            thresh2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
        self.thresh2 = thresh2
        i = 0
        m = []
        for contour in contours:  
            if i == 0: 
                i = 1
                continue
            M = cv2.moments(contour) 
            if M['m00'] != 0.0: 
                x = int(M['m10']/M['m00']) 
                y = int(M['m01']/M['m00']) 
                if len(m)<self.NB_SHAPES:
                    m.append([x,y])
        th3 = cv2.adaptiveThreshold(self.thresh2,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
                                    cv2.THRESH_BINARY,11,2)
        corners = cv2.goodFeaturesToTrack(th3, self.NB_CORNERS, 0.01, 45) 
        corners = np.int0(corners) 
        *** CODE TO PUT THE CORNERS AWAY FROM THE OBSTACLES ***

In [ ]:
def trace_contours(self):
        a = int(np.size(self.cornerss)/2)
        b = int(np.size(self.m_cor)/2)
        for i in range (0,a):
            for j in range (0,b):
                if i != j:
                    if self.m_cor[i] == self.m_cor[j]:
                        line = np.transpose(np.array(draw.line(self.cornerss[i][0], self.cornerss[i][1], self.cornerss[j][0], self.cornerss[j][1])))
                        data = self.thresh2[line[:, 1], line[:, 0]]
                        if np.size(np.where(abs(np.diff(data))>0)[0]) <= 2:
                            if np.mean(data) > self.mean_value_along_line:
                                cv2.line(self.thresh2, self.cornerss[i], self.cornerss[j], self.bluepx, 4)

In [ ]:
def compute_dist_mx(self,robot):
        s = int(((np.size(self.cor))/2)+2) 
        dist_mx = np.zeros((s,s))
        for i in range(1,s):
            if i == (s-1):
                line = np.transpose(np.array(draw.line(self.x_back,self.y_back, self.x_goal,self.y_goal)))
                data = self.thresh2[line[:, 1], line[:, 0]]
                if np.size(np.where(abs(np.diff(data))>0)[0]) <= 3 : 
                    dist_mx[i][0] = np.sqrt(np.power((self.x_goal-self.x_back),2)+np.power((self.y_goal-self.y_back),2))
                continue 
            line = np.transpose(np.array(draw.line(self.x_back,self.y_back,self.cor[i-1][0],self.cor[i-1][1])))
            data = self.thresh2[line[:, 1], line[:, 0]]
            if np.size(np.where(abs(np.diff(data))>0)[0]) <= 3 : 
                dist_mx[i][0] = np.sqrt(np.power((self.cor[i-1][0]-self.x_back),2)+np.power((self.cor[i-1][1]-self.y_back),2))
        for i in range(0,(s-2)):
            for j in range(0,s):
                if j == (i+1):
                    continue 
                if j == 0 : 
                    line = np.transpose(np.array(draw.line(self.x_back,self.y_back, self.cor[i][0],self.cor[i][1])))
                    data = self.thresh2[line[:, 1], line[:, 0]]
                    if np.size(np.where(abs(np.diff(data))>0)[0]) <= 3 : 
                        dist_mx[j][i+1] = np.sqrt(np.power((self.cor[i][0]-self.x_back),2)+np.power((self.cor[i][1]-self.y_back),2))
                    continue 
                if j == int(s-1):
                    line = np.transpose(np.array(draw.line(self.x_goal,self.y_goal, self.cor[i][0],self.cor[i][1])))
                    data = self.thresh2[line[:, 1], line[:, 0]]
                    if np.size(np.where(abs(np.diff(data))>0)[0]) <= 3 : 
                        dist_mx[j][i+1] = np.sqrt(np.power((self.cor[i][0]-self.x_goal),2)+np.power((self.cor[i][1]-self.y_goal),2))
                    continue
                line = np.transpose(np.array(draw.line(self.cor[j-1][0],self.cor[j-1][1],self.cor[i][0],self.cor[i][1])))
                data = self.thresh2[line[:, 1], line[:, 0]]
                if np.size(np.where(abs(np.diff(data))>0)[0]) <= 3 : 
                    dist_mx[j][i+1] = np.sqrt(np.power((self.cor[i][0]-self.cor[j-1][0]),2)+np.power((self.cor[i][1]-self.cor[j-1][1]),2))
        for i in range(0,(s-1)):
            if i == 0:
                line = np.transpose(np.array(draw.line(self.x_back,self.y_back, self.x_goal,self.y_goal)))
                data = self.thresh2[line[:, 1], line[:, 0]]
                if np.size(np.where(abs(np.diff(data))>0)[0]) <= 3 : 
                    dist_mx[0][s-1] = np.sqrt(np.power((self.x_goal-self.x_back),2)+np.power((self.y_goal-self.y_back),2))
                continue 
            line = np.transpose(np.array(draw.line(self.x_goal,self.y_goal,self.cor[i-1][0],self.cor[i-1][1])))
            data = self.thresh2[line[:, 1], line[:, 0]]
            if np.size(np.where(abs(np.diff(data))>0)[0]) <= 3 : 
                dist_mx[i][s-1] = np.sqrt(np.power((self.cor[i-1][0]-self.x_goal),2)+np.power((self.cor[i-1][1]-self.y_goal),2))
        self.dist_mx = dist_mx
        robot.setDistMx(dist_mx)
        robot.setCor(self.cor)
        robot.setS(s)

**Discussion of the flaws of the computer vision part**

- The color filtering is fun and useful but with hsv space and masking, we explore a "range" of values (for example "reddish" values). In noisy pictures, if the range is too large, we will detect a little bit of this color somewhere it should not be, and on the contrary if the range is too small we will not detect the target, or we will detect it in one lightning condition and not the other. To have an optimal color detection, we should have "perfect" pictures which, in reality, do not exist. It is therefore an issue to be dealt with. In our case, once the picture was well filtered and the mask were adapted to the colors we used, it worked well in most situations. 

- The functions implementing the distance matrix, putting away corners and thresholding along lines have been computed fully "by hand" and are surely not optimized. Using existing libraries and tools, we could surely optimize those functions and clean them out.

- The visibility part is very sensitive to lightning conditions 

- Our setup and code yould not work with faded colors, bright light spot somewhere on the frame, artificial colored light or unidentified obstacles.

- Our approach does not support round shaped obstacles 

- The algorithm needs to be provided the exact number of shapes and corners before the execution. Those variables can be easily modified in the code. 

**Implementation of Vision in the main**

A new instance of vision Class is created, along with a capture of the video taken by the camera. In the infinity loop, the vision functions are always called first as those values are needed for the rest of the program. It is implememted as follows:

- vision Class instance initialized outside the while loop at the beginning of the execution.

- video captured from the camera.

In the infinity loop:
- Every time, we capture a new frame, find the goal and thymio position and angle. 

- We only compute the contours and corners of the obstacles once.

- the distance matrix for the visibility graph (+ path recomputing discussed in the global navigation part) is computed the first time we execute the loop, plus everytime we are in a kidnapping situation. 

In [ ]:
#Classes initialization
robot=Thymio() # Set Thym as class Thymio as initialization before the while
KF = KalmanFilter()
vision = Vision()
global_nav = Global_Nav()

#Video capturing 
cap = cv2.VideoCapture(1)

a = 0
while(1) :
    a = a + 1
    #VISION
    vision.capture_image(cap)
    vision.find_goal_pos()
    vision.find_start_pos(robot,a)
    vision.find_angle(robot)
    if (a == 1) : 
        vision.find_corners()
        vision.trace_contours()
    if ((robot.vision == 1) & (a == 1))or((a != 1) & (robot.kidnap == True) & (robot.vision == 1)):
        vision.compute_dist_mx(robot)
        global_nav.dijkstra(robot)
        global_nav.extract_path(robot)
        if robot.kidnap == True:
            robot.kidnap = False
    robot.vision = True
    #END VISION 

## 3.2. <a id='toc3_2_'></a>[Global Navigation](#toc0_)

<u>**Idea**</u>

The **Global Navigation** module goal is to provide apath for the motion control module, based on the informations provided by the vision module. We inplemented two different global navigation methods that both ended working, but do not use the same setup and variables. Here an explanation about visibility graph method will be given. We use the distance matrix between the points provided in the robot class instance and compute the best path using Dijkstra's algorithm in the function *dijkstra* bqsed on the pseudo-code we found here: (source)

  
Difference of optimality between Astar and Visibility 

Pictures 


In [ ]:
def dijkstra(self,robot):

        self.dist_mx = robot.getDistMx()
        self.s = robot.getS()
        graph = np.zeros((3,self.s))
        graph[0] = 3000 
        graph[0][0] = 0
        graph[2] = 42
        result = 0
        current_node = 0
        a = 0
        while 1:
            b = 0
            for i in range (0,self.s):
                if graph[1][i] == 0: 
                    b = b + 1
                    if graph[0][i] < graph[0][result]:
                        result = i
                    if b == 1:
                        result = i
            current_node = result 
            graph[1][current_node] = 1
            for i in range (0,self.s):
                if self.dist_mx[i][current_node] != 0:
                    if graph[1][i] == 0: 
                        new_score = graph[0][current_node] + self.dist_mx[i][current_node]
                        if new_score < graph[0][i]:
                            graph[0][i] = new_score 
                            graph[2][i] = current_node 
            if current_node == int(self.s-1):
                break
            for i in range (0,self.s):
                if graph[1][i] == 0: 
                    if graph[0][i] < graph[0][result]:
                        result = i
                        if graph[0][result] == 3000:
                            break

        self.graph = graph 
        self.current_node = current_node 

In [ ]:
def extract_path(self,robot):
        cor = robot.getCor()
        path = [self.current_node]
        node = self.current_node
        while node != 0:
            path.append(int(self.graph[2][node]))
            node = path[-1]
        path = np.flip(path)
        path_coord = [[robot.pos_X,robot.pos_Y]]
        a = 0
        for p in path : 
            a = a + 1 
            if a != 1:
                if a != np.size(path):
                    i = int(p-1)
                    path_coord.append(cor[i])
        path_coord.append([robot.goal_X,robot.goal_Y])
        robot.path = path_coord
        if len(path_coord)>1:
            robot.setAngle()
        self.path = path 
        self.path_coord = path_coord